In [1]:
import numpy as np
import itertools
import math
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
import heapq
import pandas as pd

In [2]:
arr = np.array([[(10,1), (80,6), (37,3), (17,1), (90,10)],
                [(31,2), (50,4), (20,2), (73,4), (89,8)]])

In [3]:
thresh = np.zeros((2, 5))
for i in range(2):
    for j in range(5):
        mult, inhabitants = arr[i, j]
        thresh[i, j] = (mult - 5 * inhabitants) / 500
with np.printoptions(precision=3, suppress=True):
    print(thresh)

[[0.01  0.1   0.044 0.024 0.08 ]
 [0.042 0.06  0.02  0.106 0.098]]


In [4]:
def maximin1():
    """Solves the maximin optimization problem for a single expedition.

    Parameters
    ----------
    
    Returns
    -------
    argmax : list of tuple
        Maximizers.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    argmax = []
    for mult, inhabitants in arr.reshape(-1, 2):
        val = mult / (inhabitants + 100) * 10000
        if math.isclose(val, max_val):
            argmax.append((mult, inhabitants))
        elif val > max_val:
            argmax = [(mult, inhabitants)]
            max_val = val
    return (argmax, max_val)

In [ ]:
maximin1()

([(np.int64(89), np.int64(8))], np.float64(8240.74074074074))

In [6]:
session = WolframLanguageSession(kernel="/Applications/Wolfram Engine.app/Contents/MacOS/WolframKernel")

def maximize2():
    """Solves the maximin optimization problem for two expeditions. Returns only one solution, there may be other.

    Parameters
    ----------
    
    Returns
    -------
    argmax1 : tuple
        First expedition.
    argmax2 : tuple
        Second expedition.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    max_mult1, max_hunt1 = None, None
    max_mult2, max_hunt2 = None, None
    for (mult1, hunt1), (mult2, hunt2) in itertools.combinations(arr.reshape((-1, 2)), 2):
        val_temp = session.evaluate(wlexpr(f'NMinimize[{{{mult1}/({hunt1} + 100*p1) + {mult2}/({hunt2} + 100*p2), 0 <= p1, 0 <= p2, p1 + p2 <= 1}}, {{p1, p2}}]'))[0]
        val = -50000 + val_temp * 10000
        if math.isclose(val, max_val):
            print("collision")
        if val > max_val:
            max_mult1, max_hunt1 = mult1, hunt1
            max_mult2, max_hunt2 = mult2, hunt2
            max_val = val
    return ((max_mult1, max_hunt1), (max_mult2, max_hunt2), max_val)

In [9]:
maximize2()

((np.int64(90), np.int64(10)),
 (np.int64(89), np.int64(8)),
 -19661.253928024653)

In [10]:
ratios = np.zeros((2, 5))
for i in range(2):
    for j in range(5):
        mult, hunt = arr[i, j]
        ratios[i, j] = mult/hunt
with np.printoptions(precision=2, suppress=True):
    print("Ratios:", ratios, sep='\n')

Ratios:
[[10.   13.33 12.33 17.    9.  ]
 [15.5  12.5  10.   18.25 11.12]]


In [11]:
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior:", shares, sep='\n')

Natural prior:
[[0.077 0.103 0.096 0.132 0.07 ]
 [0.12  0.097 0.077 0.141 0.086]]


In [12]:
ratios = np.zeros((2, 5))
for i in range(2):
    for j in range(5):
        mult, hunt = arr[i, j]
        ratios[i, j] = mult/hunt
ratios = ratios**5
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior with exponent 5:", shares, sep='\n')

Natural prior with exponent 5:
[[0.017 0.073 0.049 0.246 0.01 ]
 [0.155 0.053 0.017 0.35  0.029]]


In [21]:
def fee(n):
    """Compute the fee for a total of n expeditions.

    Parameters
    ----------
    n : int
        Number of expeditions.
    
    Returns
    -------
    float
        Fee.
    """
    if n == 1:
        return 0
    if n == 2:
        return -50000

def payoff(mults, hunts, shares):
    """Compute the final profit after the expeditions.

    Parameters
    ----------
    mults : list of int
        Multipliers for each destination.
    hunts : list of int
        Hunters for each destination.
    shares : list of int
        Shares for each destination.
    
    Returns
    -------
    float
        Profit.
    """
    return 10000 * sum([mult/(hunt + 100*share) for (mult, hunt, share) in zip(mults, hunts, shares)]) + fee(len(mults))

def maximize_prior_top(shares, k):
    """Given the prior, compute solutions that yield top k profits.

    Parameters
    ----------
    shares : list of int
        Shares for each destination.
    k : int
        Number of solutions
    
    Returns
    -------
    list of tuple
        Top k profits and optimal expeditions.
    """
    datas = [(mult, hunt, share) for ((mult, hunt), share) in zip(arr.reshape((-1, 2)), shares.reshape(-1))]
    print(datas)
    heap = []
    iterables = [itertools.combinations(datas, n_exp) for n_exp in range(1, 3)]
    for (i, data) in enumerate(itertools.chain.from_iterable(iterables)):
        mults = [tupl[0] for tupl in data]
        hunts = [tupl[1] for tupl in data]
        shares = [tupl[-1] for tupl in data]
        val = payoff(mults, hunts, shares)
        expeditions = list(zip(mults, hunts))
        if i < k:
            heapq.heappush(heap, (val, expeditions))
        elif val > heap[0][0]:
            heapq.heappop(heap)
            heapq.heappush(heap, (val, expeditions))
    return sorted(heap, reverse=True)

In [24]:
for i in range(10):
    shares = ratios**i
    shares = shares / np.sum(shares)
    res = maximize_prior_top(shares, 1)
    
    # convert res values to int
    res = [(int(x[0]), [(int(y[0]), int(y[1])) for y in x[1]]) for x in res]
    print("Exponent:", i, "Profit:", f"{res[0][0]:.2f}", "Optimal expeditions:", res[0][1])

[(np.int64(10), np.int64(1), np.float64(0.1)), (np.int64(80), np.int64(6), np.float64(0.1)), (np.int64(37), np.int64(3), np.float64(0.1)), (np.int64(17), np.int64(1), np.float64(0.1)), (np.int64(90), np.int64(10), np.float64(0.1)), (np.int64(31), np.int64(2), np.float64(0.1)), (np.int64(50), np.int64(4), np.float64(0.1)), (np.int64(20), np.int64(2), np.float64(0.1)), (np.int64(73), np.int64(4), np.float64(0.1)), (np.int64(89), np.int64(8), np.float64(0.1))]
Exponent: 0 Profit: 52142.00 Optimal expeditions: [(80, 6), (73, 4)]
[(np.int64(10), np.int64(1), np.float64(0.017299829010239777)), (np.int64(80), np.int64(6), np.float64(0.07290133706372649)), (np.int64(37), np.int64(3), np.float64(0.04936784357997613)), (np.int64(17), np.int64(1), np.float64(0.24563283318992019)), (np.int64(90), np.int64(10), np.float64(0.010215376032256486)), (np.int64(31), np.int64(2), np.float64(0.15477481781510471)), (np.int64(50), np.int64(4), np.float64(0.052794888336913384)), (np.int64(20), np.int64(2), np